# バッチ処理
MNISTデータセットを扱ったニューラルネットワークにおける、入力データと重みパラメータの「形状」に注意して再度実装を試みる（実際、データの中身はブラックボックスで、本当に計算可能であるのかを検証する必要は、他の研究でも出てくるだろう）。

In [1]:
# まずは準備

import sys, os
sys.path.append(os.pardir)
import pickle
from dataset.mnist import load_mnist

import numpy as np

def sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return y

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    exp_sum = np.sum(exp_a)
    y = exp_a / exp_sum
    return y

def get_data():
    (x_train, t_train), (x_test, t_test) = (
        load_mnist(normalize = True, flatten = True, one_hot_label = False)
    )
    return x_test, t_test

def init_network():
    with open("G:/マイドライブ/勉強系/心理学/森口研究室\DeepLearning_from-Zero\sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    
    return network

def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

<>:28: SyntaxWarning: invalid escape sequence '\D'
<>:28: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Cyril\AppData\Local\Temp\ipykernel_7228\3270765219.py:28: SyntaxWarning: invalid escape sequence '\D'
  with open("G:/マイドライブ/勉強系/心理学/森口研究室\DeepLearning_from-Zero\sample_weight.pkl", "rb") as f:


インタプリタを用いて実装する様子をnotebookで再現する。

In [2]:
x, _ = get_data()

In [3]:
network = init_network()

In [4]:
W1, W2, W3 = network["W1"], network["W2"], network["W3"]

In [5]:
x.shape

(10000, 784)

In [6]:
x[0].shape

(784,)

In [7]:
length_check = []
for i in x:
    length_check.append(i.shape == (784,))

all(length_check)

True

In [8]:
W1.shape

(784, 50)

In [9]:
W2.shape

(50, 100)

In [10]:
W3.shape

(100, 10)

確かに、整合性が取れている（まぁ、とれてなかったらエラーを吐くから、こんなことをしなくてもいいんだけどね）

では、バッチ処理（まとまりのあるデータにすること）による実装を行う。

In [11]:
x, t = get_data()
network = init_network()

batch_size = 100    # バッチの数
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i + batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 1)
    accuracy_cnt += np.sum(p == t[i: i + batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


In [12]:
# そもそも、バッチ処理は初めからされているし、どちらも処理時間は変わらないから、これでいいんじゃないか？
x, t = get_data()
network = init_network()

y = predict(network, x)
p = np.argmax(y, axis = 1)

accuracy_cnt = np.sum(p == t)
print(float(accuracy_cnt) / len(x))

0.9352


In [13]:
# ステップ関数だとどうなるんだろう？

import sys, os
sys.path.append(os.pardir)
import pickle
from dataset.mnist import load_mnist

import numpy as np

def step_function(x):
    return np.array(x > 0, dtype = int)

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    exp_sum = np.sum(exp_a)
    y = exp_a / exp_sum
    return y

def get_data():
    (x_train, t_train), (x_test, t_test) = (
        load_mnist(normalize = True, flatten = True, one_hot_label = False)
    )
    return x_test, t_test

def init_network():
    with open("G:/マイドライブ/勉強系/心理学/森口研究室\DeepLearning_from-Zero\sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    
    return network

def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]

    a1 = np.dot(x, W1) + b1
    z1 = step_function(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = step_function(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

<>:27: SyntaxWarning: invalid escape sequence '\D'
<>:27: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Cyril\AppData\Local\Temp\ipykernel_7228\589323436.py:27: SyntaxWarning: invalid escape sequence '\D'
  with open("G:/マイドライブ/勉強系/心理学/森口研究室\DeepLearning_from-Zero\sample_weight.pkl", "rb") as f:


In [14]:
x, t = get_data()
network = init_network()

y = predict(network, x)
p = np.argmax(y, axis = 1)

accuracy_cnt = np.sum(p == t)
print(float(accuracy_cnt) / len(x))

0.9182


シグモイド関数ではなく、ステップ関数を用いたら、先ほどよりも識別精度が低くなった！